In [36]:

import pickle
import numpy as np
import pandas as pd


In [37]:
from tensorflow.keras.models import load_model

In [38]:
## load the tarined model , scala, pickle, onehot
model=load_model('model.h5')

## Load the scaler and encodder
with open("onehot_encoder_geo.pkl", "rb") as f:
    label_encoder_geo = pickle.load(f)

with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)
    
with open("label_encoder_gender.pkl", "rb") as f:
    label_encoder_gender = pickle.load(f)

In [49]:
## Example input data for prediction
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender':'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [50]:
## one hot geo_encoded
geo_encoded = label_encoder_geo.transform([[input_data["Geography"]]]).toarray() ## i use this because it returns a numpy array
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\dell\anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [51]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [52]:
## Encode Categorial variables
input_df["Gender"] = label_encoder_gender.transform(input_df["Gender"])
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [53]:
## Concatenate one hot encoded geo columns with input data
input_df=pd.concat([input_df.drop("Geography",axis=1), geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [54]:
## Scaling the data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.47154541,  0.90911166,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184]])

In [55]:
## Prediction Churn
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 635ms/step


array([[0.01855083]], dtype=float32)

In [57]:
prediction_proba = prediction[0][0]  # Assuming the model outputs a single value for churn probability
prediction_proba

np.float32(0.01855083)

In [58]:
if prediction_proba > 0.5:
    print("The customer is likely to churn with a probability of {:.2f}%".format(prediction_proba * 100))
else:
    print("The customer is not likely to churn with a probability of {:.2f}%".format((1 - prediction_proba) * 100))

The customer is not likely to churn with a probability of 98.14%
